<a href="https://colab.research.google.com/github/leandrorondini/PowerBI-DataScience/blob/master/Relatorio_acoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CÉLULA 1 - BIBLIOTECAS CORRETAS
!pip install yfinance pandas-ta

import yfinance as yf
import pandas as pd
import pandas_ta as ta
import smtplib
from email.mime.text import MIMEText  # ✅ CORRETO
from email.mime.multipart import MIMEMultipart  # ✅ CORRETO
from datetime import datetime, timedelta
import time

print("✅ Todas as bibliotecas carregadas!")
print("Agora execute a próxima célula...")

✅ Todas as bibliotecas carregadas!
Agora execute a próxima célula...


In [ ]:
# CÉLULA 2 - LISTA DE AÇÕES
acoes_ibovespa = [
    'ABEV3.SA', 'ALPA4.SA', 'AMER3.SA', 'ASAI3.SA', 'AZUL4.SA', 'B3SA3.SA', 'BBAS3.SA', 'BBDC3.SA', 'BBDC4.SA',
    'BBSE3.SA', 'BEEF3.SA', 'BPAC11.SA', 'BRAP4.SA', 'BRDT3.SA', 'BRFS3.SA', 'BRKM5.SA', 'BRML3.SA', 'CASH3.SA',
    'CCRO3.SA', 'CIEL3.SA', 'CMIG4.SA', 'COGN3.SA', 'CPFE3.SA', 'CPLE6.SA', 'CSAN3.SA', 'CSNA3.SA', 'CVCB3.SA',
    'CYRE3.SA', 'ECOR3.SA', 'EGIE3.SA', 'ELET3.SA', 'ELET6.SA', 'EMBR3.SA', 'ENBR3.SA', 'ENEV3.SA', 'ENGI11.SA',
    'EQTL3.SA', 'EZTC3.SA', 'FLRY3.SA', 'GGBR4.SA', 'GOAU4.SA', 'GOLL4.SA', 'HAPV3.SA', 'HYPE3.SA', 'IGTA3.SA',
    'IRBR3.SA', 'ITSA4.SA', 'ITUB4.SA', 'JBSS3.SA', 'JHSF3.SA', 'KLBN11.SA', 'LREN3.SA', 'LWSA3.SA', 'MGLU3.SA',
    'MRFG3.SA', 'MRVE3.SA', 'MULT3.SA', 'NTCO3.SA', 'PCAR3.SA', 'PETR3.SA', 'PETR4.SA', 'PRIO3.SA', 'QUAL3.SA',
    'RADL3.SA', 'RAIL3.SA', 'RDOR3.SA', 'RENT3.SA', 'RRRP3.SA', 'SANB11.SA', 'SBSP3.SA', 'SLCE3.SA', 'SOMA3.SA',
    'SUZB3.SA', 'TAEE11.SA', 'TIMS3.SA', 'TOTS3.SA', 'UGPA3.SA', 'USIM5.SA', 'VALE3.SA', 'VBBR3.SA', 'VIVT3.SA',
    'WEGE3.SA', 'YDUQ3.SA'
]

print(f"📊 {len(acoes_ibovespa)} ações carregadas")

📊 83 ações carregadas


In [ ]:
# CÉLULA 3 - ANALISADOR DE CANDLESTICKS
class AnalisadorCandlesticks:
    def __init__(self):
        self.padroes_alta = {
            'martelo': self._detectar_martelo,
            'martelo_invertido': self._detectar_martelo_invertido,
            'engolfo_altista': self._detectar_engolfo_altista,
            'estrela_amanhecer': self._detectar_estrela_amanhecer,
            'linha_perfurada': self._detectar_linha_perfurada,
            'tres_soldados_brancos': self._detectar_tres_soldados_brancos
        }

        self.confiabilidade_padroes = {
            'martelo': 0.65,
            'martelo_invertido': 0.60,
            'engolfo_altista': 0.75,
            'estrela_amanhecer': 0.70,
            'linha_perfurada': 0.68,
            'tres_soldados_brancos': 0.80
        }

    def _detectar_martelo(self, df):
        if len(df) < 2:
            return False

        atual = df.iloc[-1]
        corpo = abs(atual['Close'] - atual['Open'])
        sombra_superior = atual['High'] - max(atual['Open'], atual['Close'])
        sombra_inferior = min(atual['Open'], atual['Close']) - atual['Low']

        return (sombra_inferior >= 2 * corpo and
                sombra_superior <= corpo * 0.3 and
                atual['Close'] > atual['Open'])

    def _detectar_martelo_invertido(self, df):
        if len(df) < 2:
            return False

        atual = df.iloc[-1]
        corpo = abs(atual['Close'] - atual['Open'])
        sombra_superior = atual['High'] - max(atual['Open'], atual['Close'])
        sombra_inferior = min(atual['Open'], atual['Close']) - atual['Low']

        return (sombra_superior >= 2 * corpo and
                sombra_inferior <= corpo * 0.3)

    def _detectar_engolfo_altista(self, df):
        if len(df) < 2:
            return False

        atual = df.iloc[-1]
        anterior = df.iloc[-2]

        return (atual['Close'] > atual['Open'] and
                anterior['Close'] < anterior['Open'] and
                atual['Open'] < anterior['Close'] and
                atual['Close'] > anterior['Open'])

    def _detectar_estrela_amanhecer(self, df):
        if len(df) < 3:
            return False

        primeiro = df.iloc[-3]
        segundo = df.iloc[-2]
        terceiro = df.iloc[-1]

        return (primeiro['Close'] < primeiro['Open'] and
                abs(segundo['Close'] - segundo['Open']) / primeiro['Close'] < 0.01 and
                terceiro['Close'] > terceiro['Open'] and
                terceiro['Open'] > segundo['High'])

    def _detectar_linha_perfurada(self, df):
        if len(df) < 2:
            return False

        anterior = df.iloc[-2]
        atual = df.iloc[-1]

        return (anterior['Close'] < anterior['Open'] and
                atual['Close'] > atual['Open'] and
                atual['Open'] < anterior['Low'] and
                atual['Close'] > (anterior['Open'] + anterior['Close']) / 2)

    def _detectar_tres_soldados_brancos(self, df):
        if len(df) < 3:
            return False

        candles = [df.iloc[-3], df.iloc[-2], df.iloc[-1]]

        for i in range(3):
            if candles[i]['Close'] <= candles[i]['Open']:
                return False
            if i > 0 and candles[i]['Open'] < candles[i-1]['Close']:
                return False

        return True

    def calcular_probabilidade_alta(self, df):
        if len(df) < 3:
            return 0.5, "Dados insuficientes", []

        probabilidade_base = 0.5
        padroes_detectados = []

        for nome_padrao, detector in self.padroes_alta.items():
            if detector(df):
                padroes_detectados.append(nome_padrao)
                confiabilidade = self.confiabilidade_padroes[nome_padrao]
                probabilidade_base += (confiabilidade - 0.5) * 0.3

        probabilidade = max(0.1, min(0.9, probabilidade_base))

        if len(df) >= 5:
            tendencia_recente = (df['Close'].iloc[-1] - df['Close'].iloc[-5]) / df['Close'].iloc[-5]
            if tendencia_recente > 0.02:
                probabilidade += 0.1
            elif tendencia_recente < -0.02:
                probabilidade -= 0.1

        if probabilidade >= 0.78:
            classificacao = "ALTA PROBABILIDADE"
        elif probabilidade >= 0.65:
            classificacao = "PROVAVEL ALTA"
        elif probabilidade >= 0.55:
            classificacao = "NEUTRO TENDENDO ALTA"
        elif probabilidade >= 0.4:
            classificacao = "NEUTRO TENDENDO BAIXA"
        else:
            classificacao = "BAIXA PROBABILIDADE"

        return probabilidade, classificacao, padroes_detectados

analisador = AnalisadorCandlesticks()
print("✅ Analisador de candlesticks carregado!")

✅ Analisador de candlesticks carregado!


In [ ]:
# CÉLULA 4 - FUNÇÃO DE ANÁLISE
def analisar_mercado(acoes):
    """Analisa todas as ações e retorna as melhores oportunidades"""
    print("🔍 Iniciando análise do mercado...")
    resultados = []

    for i, ticker in enumerate(acoes):
        try:
            acao = yf.Ticker(ticker)
            hist = acao.history(period='30d')

            if len(hist) < 5:
                continue

            preco_atual = hist['Close'].iloc[-1]
            preco_anterior = hist['Close'].iloc[-2]
            variacao = ((preco_atual - preco_anterior) / preco_anterior) * 100

            probabilidade, classificacao, padroes = analisador.calcular_probabilidade_alta(hist)

            info_acao = {
                'ticker': ticker,
                'preco_atual': round(preco_atual, 2),
                'variacao_dia': round(variacao, 2),
                'probabilidade_alta': round(probabilidade * 100, 1),
                'classificacao': classificacao,
                'padroes_detectados': ', '.join(padroes) if padroes else 'Nenhum'
            }

            resultados.append(info_acao)

            if (i + 1) % 10 == 0:
                print(f"📊 Analisadas {i + 1}/{len(acoes)} ações...")

            time.sleep(0.1)

        except Exception as e:
            continue

    # Ordenar por probabilidade
    resultados_ordenados = sorted(resultados, key=lambda x: x['probabilidade_alta'], reverse=True)
    top_10 = resultados_ordenados[:10]

    print(f"✅ Análise concluída! {len(resultados)} ações processadas")
    return top_10

print("✅ Função de análise carregada!")

✅ Função de análise carregada!


In [ ]:
# CÉLULA 5 - ENVIAR RELATÓRIO POR EMAIL (EXECUTE ESTA!)

# Configurações do Email (MODIFIQUE A SENHA)
email_from = "leandrorondini@gmail.com"
email_to = "leandrorondini@gmail.com"
email_password = "ylsr yrji knfz soos"  # ⚠️ COLOQUE SUA SENHA DE APP AQUI!

def enviar_relatorio_email():
    """Função principal que executa tudo e envia o email"""

    print("🚀 INICIANDO PROCESSO COMPLETO...")
    print("1. Analisando ações da Bovespa...")

    # Executar análise
    top_10 = analisar_mercado(acoes_ibovespa)

    print("2. Criando relatório...")

    # Criar conteúdo do email
    data_atual = datetime.now().strftime("%d/%m/%Y %H:%M")

    html_content = f"""
    <html>
    <head>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            .header {{ background: #2E86AB; color: white; padding: 20px; border-radius: 10px; }}
            .ticker {{ background: #F8F9FA; padding: 15px; margin: 10px 0; border-radius: 8px; border-left: 5px solid #2E86AB; }}
            .alta {{ border-left-color: #28A745 !important; }}
            .media {{ border-left-color: #FFC107 !important; }}
            .probabilidade {{ font-weight: bold; color: #28A745; }}
            .variacao-positiva {{ color: #28A745; }}
            .variacao-negativa {{ color: #DC3545; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>📈 RELATÓRIO BOVESPA - {data_atual}</h1>
            <p>Top 10 Ações com Maior Probabilidade de Alta</p>
        </div>

        <h2>🎯 Melhores Oportunidades do Dia</h2>
    """

    for i, acao in enumerate(top_10, 1):
        variacao_class = "variacao-positiva" if acao['variacao_dia'] > 0 else "variacao-negativa"
        classe_ticker = "alta" if acao['probabilidade_alta'] > 70 else "media"

        html_content += f"""
        <div class="ticker {classe_ticker}">
            <h3>#{i} - {acao['ticker']}</h3>
            <p><strong>Preço:</strong> R$ {acao['preco_atual']}</p>
            <p><strong>Variação do Dia:</strong> <span class="{variacao_class}">{acao['variacao_dia']}%</span></p>
            <p><strong>Probabilidade de Alta:</strong> <span class="probabilidade">{acao['probabilidade_alta']}%</span></p>
            <p><strong>Classificação:</strong> {acao['classificacao']}</p>
            <p><strong>Padrões Detectados:</strong> {acao['padroes_detectados']}</p>
        </div>
        """

    html_content += """
        <br>
        <hr>
        <p><em>Relatório gerado automaticamente - Análise baseada em padrões de candlestick</em></p>
    </body>
    </html>
    """

    print("3. Enviando email...")

    try:
        # Configurar servidor SMTP do Gmail
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()

        # ⚠️ IMPORTANTE: Use a senha de app do Gmail aqui
        server.login(email_from, email_password)

        # Criar mensagem
        msg = MIMEMultipart()
        msg['From'] = email_from
        msg['To'] = email_to
        msg['Subject'] = f"📊 Relatório Bovespa - {datetime.now().strftime('%d/%m/%Y')}"

        # Adicionar conteúdo HTML
        msg.attach(MIMEText(html_content, 'html'))

        # Enviar email
        server.send_message(msg)
        server.quit()

        print("✅ EMAIL ENVIADO COM SUCESSO!")
        print(f"📧 Para: {email_to}")
        print(f"📊 {len(top_10)} oportunidades identificadas")

        # Mostrar preview no Colab
        print("\n📋 PREVIEW DO RELATÓRIO:")
        for i, acao in enumerate(top_10, 1):
            print(f"{i}. {acao['ticker']} - {acao['probabilidade_alta']}% - {acao['classificacao']}")

    except Exception as e:
        print(f"❌ ERRO AO ENVIAR EMAIL: {e}")
        print("💡 Verifique a senha de app do Gmail!")

# EXECUTAR TUDO - ESTA LINHA INICIA O PROCESSO COMPLETO
enviar_relatorio_email()

🚀 INICIANDO PROCESSO COMPLETO...
1. Analisando ações da Bovespa...
🔍 Iniciando análise do mercado...
📊 Analisadas 10/83 ações...


ERROR:yfinance:$BRDT3.SA: possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$BRFS3.SA: possibly delisted; no price data found  (period=30d)
ERROR:yfinance:$BRML3.SA: possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$CCRO3.SA: possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$CIEL3.SA: possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")


📊 Analisadas 30/83 ações...


ERROR:yfinance:$ENBR3.SA: possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")


📊 Analisadas 40/83 ações...


...

📊 Analisadas 60/83 ações...


ERROR:yfinance:$RRRP3.SA: possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")


📊 Analisadas 70/83 ações...


ERROR:yfinance:$SOMA3.SA: possibly delisted; no price data found  (period=30d) (Yahoo error = "No data found, symbol may be delisted")


📊 Analisadas 80/83 ações...
✅ Análise concluída! 71 ações processadas
2. Criando relatório...
3. Enviando email...
✅ EMAIL ENVIADO COM SUCESSO!
📧 Para: leandrorondini@gmail.com
📊 10 oportunidades identificadas

📋 PREVIEW DO RELATÓRIO:
1. EQTL3.SA - 69.0% - PROVAVEL ALTA
2. PETR4.SA - 69.0% - PROVAVEL ALTA
3. PRIO3.SA - 69.0% - PROVAVEL ALTA
4. TAEE11.SA - 69.0% - PROVAVEL ALTA
5. BBDC3.SA - 66.0% - PROVAVEL ALTA
6. BBDC4.SA - 66.0% - PROVAVEL ALTA
7. MULT3.SA - 66.0% - PROVAVEL ALTA
8. SANB11.SA - 66.0% - PROVAVEL ALTA
9. ALPA4.SA - 60.0% - NEUTRO TENDENDO ALTA
10. AMER3.SA - 60.0% - NEUTRO TENDENDO ALTA


In [ ]:
# CÉLULA 6 - AGENDAMENTO AUTOMÁTICO NO COLAB
def agendar_execucao():
    print("⏰ CONFIGURANDO AGENDAMENTO AUTOMÁTICO")
    print("\n📋 INSTRUÇÕES:")
    print("1. Clique nos 3 pontinhos (⋮) no canto superior direito")
    print("2. Selecione 'Agendar execução'")
    print("3. Configure para rodar TODO O NOTEBOOK")
    print("4. Escolha o horário: 10:00 (após a abertura do mercado)")
    print("5. Salve a agenda")
    print("\n🎯 DICA: Agende para seg/sex às 10:00 para análise diária")

    # Teste rápido para verificar se tudo está funcionando
    print("\n🧪 TESTE RÁPIDO DO SISTEMA:")
    try:
        # Testar com apenas 5 ações para verificação rápida
        acoes_teste = ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA']
        resultado_teste = analisar_mercado(acoes_teste)
        if resultado_teste:
            print("✅ Sistema funcionando perfeitamente!")
            print(f"📊 {len(resultado_teste)} ações analisadas no teste")
        else:
            print("⚠️ Sistema funcionando, mas sem resultados no teste")
    except Exception as e:
        print(f"❌ Erro no teste: {e}")

agendar_execucao()

⏰ CONFIGURANDO AGENDAMENTO AUTOMÁTICO

📋 INSTRUÇÕES:
1. Clique nos 3 pontinhos (⋮) no canto superior direito
2. Selecione 'Agendar execução'
3. Configure para rodar TODO O NOTEBOOK
4. Escolha o horário: 10:00 (após a abertura do mercado)
5. Salve a agenda

🎯 DICA: Agende para seg/sex às 10:00 para análise diária

🧪 TESTE RÁPIDO DO SISTEMA:
🔍 Iniciando análise do mercado...
✅ Análise concluída! 3 ações processadas
✅ Sistema funcionando perfeitamente!
📊 3 ações analisadas no teste


In [ ]:
# CÉLULA 7 - SISTEMA DE CONFIRMAÇÃO POR VOLUME
print("📊 CONFIGURANDO SISTEMA DE CONFIRMAÇÃO AVANÇADO")

class ConfirmadorVolume:
    def __init__(self):
        self.volume_minimo = 1.2  # 20% acima da média

    def analisar_volume_padrao(self, df, padrao_nome):
        """Analisa volume para confirmar padrões de candlestick"""
        if len(df) < 21:
            return False, 0, 0, 0

        volume_atual = df['Volume'].iloc[-1]
        volume_medio = df['Volume'].tail(20).mean()

        if volume_medio == 0:
            return False, volume_atual, volume_medio, 0

        variacao_percentual = (volume_atual - volume_medio) / volume_medio

        # Regras específicas por padrão
        regras_volume = {
            'engolfo_altista': variacao_percentual > 0.3,      # +30%
            'tres_soldados_brancos': variacao_percentual > 0.2, # +20%
            'estrela_amanhecer': variacao_percentual > 0.1,    # +10%
            'martelo': variacao_percentual > 0,                # Qualquer aumento
            'linha_perfurada': variacao_percentual > 0.1,      # +10%
        }

        confirmado = regras_volume.get(padrao_nome, variacao_percentual > 0)

        return confirmado, volume_atual, volume_medio, variacao_percentual

# Instanciar o confirmador
confirmador_volume = ConfirmadorVolume()
print("✅ Sistema de confirmação por volume carregado!")

📊 CONFIGURANDO SISTEMA DE CONFIRMAÇÃO AVANÇADO
✅ Sistema de confirmação por volume carregado!


In [ ]:
# CÉLULA 8 - SISTEMA DE MÉDIAS MÓVEIS
print("📈 CONFIGURANDO ANÁLISE DE MÉDIAS MÓVEIS")

class AnalisadorMediasMoveis:
    def __init__(self):
        self.periodos = [9, 21, 50]  # Médias curta, média e longa

    def analisar_medias(self, df):
        """Analisa alinhamento das médias móveis"""
        if len(df) < max(self.periodos):
            return False, {}, "Dados insuficientes"

        resultados = {}
        preco_atual = df['Close'].iloc[-1]

        # Calcular médias
        for periodo in self.periodos:
            media = df['Close'].rolling(periodo).mean().iloc[-1]
            resultados[f'MA{periodo}'] = round(media, 2)

        # Análise de tendência
        ma9, ma21, ma50 = resultados['MA9'], resultados['MA21'], resultados['MA50']

        # Cruzamento dourado
        cruzamento_dourado = ma9 > ma21 > ma50

        # Alinhamento altista
        alinhamento_altista = preco_atual > ma9 > ma21 > ma50

        # Suporte na média
        suporte_ma21 = df['Low'].iloc[-1] > ma21

        # Classificação
        if cruzamento_dourado and alinhamento_altista:
            classificacao = "TENDÊNCIA FORTE DE ALTA"
            pontos = 3
        elif alinhamento_altista:
            classificacao = "TENDÊNCIA DE ALTA"
            pontos = 2
        elif preco_atual > ma9:
            classificacao = "TENDÊNCIA LEVE DE ALTA"
            pontos = 1
        else:
            classificacao = "TENDÊNCIA NEUTRA/BAIXA"
            pontos = 0

        return pontos > 0, resultados, classificacao, pontos

# Instanciar analisador
analisador_medias = AnalisadorMediasMoveis()
print("✅ Sistema de médias móveis carregado!")

📈 CONFIGURANDO ANÁLISE DE MÉDIAS MÓVEIS
✅ Sistema de médias móveis carregado!


In [ ]:
# CÉLULA 9 - SISTEMA DE MOMENTUM (RSI)
print("🔮 CONFIGURANDO INDICADORES DE MOMENTUM")

class AnalisadorMomentum:
    def __init__(self):
        self.rsi_sobrevenda = 30
        self.rsi_sobrecompra = 70

    def calcular_rsi(self, df, periodo=14):
        """Calcula RSI e analisa condições"""
        if len(df) < periodo + 1:
            return None, "Dados insuficientes"

        # Calcular RSI manualmente para mais controle
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=periodo).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periodo).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))

        rsi_atual = rsi.iloc[-1]
        rsi_anterior = rsi.iloc[-2] if len(rsi) > 1 else rsi_atual

        # Análise do RSI
        if pd.isna(rsi_atual):
            return None, "RSI não calculado"

        condicoes = {
            'sobrevenda': rsi_atual < self.rsi_sobrevenda,
            'sobrecompra': rsi_atual > self.rsi_sobrecompra,
            'saindo_sobrevenda': rsi_anterior < self.rsi_sobrevenda and rsi_atual > self.rsi_sobrevenda,
            'zona_neutra': self.rsi_sobrevenda <= rsi_atual <= self.rsi_sobrecompra,
            'crescendo': rsi_atual > rsi_anterior
        }

        # Pontuação baseada nas condições
        pontos = 0
        if condicoes['saindo_sobrevenda']:
            pontos += 3
            classificacao = "FORTE MOMENTUM - SAINDO DE SOBREVENDA"
        elif condicoes['sobrevenda']:
            pontos += 2
            classificacao = "MOMENTUM POSITIVO - EM SOBREVENDA"
        elif condicoes['zona_neutra'] and condicoes['crescendo']:
            pontos += 1
            classificacao = "MOMENTUM NEUTRO POSITIVO"
        else:
            classificacao = "MOMENTOM NEUTRO/NEGATIVO"

        return round(rsi_atual, 2), classificacao, pontos, condicoes

# Instanciar analisador
analisador_momentum = AnalisadorMomentum()
print("✅ Sistema de momentum (RSI) carregado!")

🔮 CONFIGURANDO INDICADORES DE MOMENTUM
✅ Sistema de momentum (RSI) carregado!


In [ ]:
# CÉLULA 10 - SISTEMA COMPLETO DE CONFIRMAÇÃO
print("🎯 CONFIGURANDO SISTEMA INTEGRADO DE CONFIRMAÇÃO")

class SistemaConfirmacaoCompleto:
    def __init__(self):
        self.pesos = {
            'volume': 0.25,
            'medias_moveis': 0.30,
            'momentum': 0.25,
            'padrao_base': 0.20
        }

    def analisar_confirmacoes(self, df, padrao_detectado, confianca_base):
        """
        Analisa todas as confirmações para um padrão detectado
        Retorna: confiança_final, detalhes_confirmacoes
        """
        print(f"🔍 Analisando confirmações para: {padrao_detectado}")

        detalhes = {}
        pontuacao_total = 0

        # 1. Confirmação por Volume
        try:
            volume_confirmado, vol_atual, vol_medio, variacao = confirmador_volume.analisar_volume_padrao(df, padrao_detectado)
            detalhes['volume'] = {
                'confirmado': volume_confirmado,
                'volume_atual': f"{vol_atual:,.0f}",
                'volume_medio': f"{vol_medio:,.0f}",
                'variacao_percentual': f"{variacao:+.1%}",
                'pontos': 1 if volume_confirmado else 0
            }
            pontuacao_total += detalhes['volume']['pontos'] * self.pesos['volume']
        except Exception as e:
            detalhes['volume'] = {'confirmado': False, 'erro': str(e), 'pontos': 0}

        # 2. Confirmação por Médias Móveis
        try:
            media_confirmado, valores_medias, classificacao_media, pontos_media = analisador_medias.analisar_medias(df)
            detalhes['medias_moveis'] = {
                'confirmado': media_confirmado,
                'valores': valores_medias,
                'classificacao': classificacao_media,
                'pontos': pontos_media
            }
            pontuacao_total += pontos_media * self.pesos['medias_moveis'] / 3.0
        except Exception as e:
            detalhes['medias_moveis'] = {'confirmado': False, 'erro': str(e), 'pontos': 0}

        # 3. Confirmação por Momentum (RSI)
        try:
            rsi_valor, classificacao_rsi, pontos_rsi, condicoes_rsi = analisador_momentum.calcular_rsi(df)
            detalhes['momentum'] = {
                'confirmado': pontos_rsi > 0,
                'rsi': rsi_valor,
                'classificacao': classificacao_rsi,
                'pontos': pontos_rsi
            }
            pontuacao_total += pontos_rsi * self.pesos['momentum'] / 3.0
        except Exception as e:
            detalhes['momentum'] = {'confirmado': False, 'erro': str(e), 'pontos': 0}

        # 4. Peso do padrão base
        pontuacao_total += confianca_base * self.pesos['padrao_base']

        # Calcular confiança final (0-100%)
        confianca_final = min(0.95, pontuacao_total)

        # Contar confirmações ativas
        confirmacoes_ativas = sum([
            detalhes['volume'].get('confirmado', False),
            detalhes['medias_moveis'].get('confirmado', False),
            detalhes['momentum'].get('confirmado', False)
        ])

        return confianca_final, detalhes, confirmacoes_ativas

# Instanciar sistema completo
sistema_confirmacao = SistemaConfirmacaoCompleto()
print("✅ Sistema integrado de confirmação carregado!")

🎯 CONFIGURANDO SISTEMA INTEGRADO DE CONFIRMAÇÃO
✅ Sistema integrado de confirmação carregado!


In [ ]:
# CÉLULA 11 - FUNÇÃO DE ANÁLISE ATUALIZADA COM CONFIRMAÇÕES
print("🔄 ATUALIZANDO FUNÇÃO PRINCIPAL DE ANÁLISE")

def analisar_mercado_com_confirmacoes(acoes):
    """Versão atualizada com sistema de confirmações"""
    print("🔍 Iniciando análise COMPLETA do mercado...")
    resultados = []

    for i, ticker in enumerate(acoes):
        try:
            acao = yf.Ticker(ticker)
            hist = acao.history(period='30d')

            if len(hist) < 21:  # Agora precisa de mais dados para as confirmações
                continue

            preco_atual = hist['Close'].iloc[-1]
            preco_anterior = hist['Close'].iloc[-2]
            variacao = ((preco_atual - preco_anterior) / preco_anterior) * 100

            # Detectar padrões primeiro
            padrao_detectado = None
            confianca_base = 0.5

            for padrao_nome, detector in analisador.padroes_alta.items():
                if detector(hist):
                    padrao_detectado = padrao_nome
                    confianca_base = analisador.confiabilidade_padroes[padrao_nome]
                    break

            # Aplicar confirmações se padrão detectado
            if padrao_detectado:
                confianca_final, detalhes_confirmacoes, num_confirmacoes = sistema_confirmacao.analisar_confirmacoes(
                    hist, padrao_detectado, confianca_base
                )
            else:
                confianca_final = 0.3  # Probabilidade baixa sem padrão
                detalhes_confirmacoes = {}
                num_confirmacoes = 0

            # Classificação baseada na confiança final
            if confianca_final >= 0.7:
                classificacao = "ALTA PROBABILIDADE CONFIRMADA"
            elif confianca_final >= 0.6:
                classificacao = "PROVÁVEL ALTA"
            elif confianca_final >= 0.5:
                classificacao = "NEUTRO TENDENDO ALTA"
            else:
                classificacao = "BAIXA PROBABILIDADE"

            info_acao = {
                'ticker': ticker,
                'preco_atual': round(preco_atual, 2),
                'variacao_dia': round(variacao, 2),
                'probabilidade_alta': round(confianca_final * 100, 1),
                'classificacao': classificacao,
                'padrao_detectado': padrao_detectado or 'Nenhum',
                'confirmacoes': num_confirmacoes,
                'detalhes_confirmacoes': detalhes_confirmacoes
            }

            resultados.append(info_acao)

            if (i + 1) % 10 == 0:
                print(f"📊 Analisadas {i + 1}/{len(acoes)} ações...")

            time.sleep(0.1)

        except Exception as e:
            continue

    # Ordenar por probabilidade final
    resultados_ordenados = sorted(resultados, key=lambda x: x['probabilidade_alta'], reverse=True)
    top_10 = resultados_ordenados[:10]

    print(f"✅ Análise COMPLETA concluída! {len(resultados)} ações processadas")
    return top_10

print("✅ Função de análise atualizada carregada!")

🔄 ATUALIZANDO FUNÇÃO PRINCIPAL DE ANÁLISE
✅ Função de análise atualizada carregada!


In [ ]:
# CÉLULA 13 - RELATÓRIO DE EMAIL ATUALIZADO COM CONFIRMAÇÕES
print("📊 CONFIGURANDO RELATÓRIO COMPLETO COM CONFIRMAÇÕES")

def enviar_relatorio_completo():
    """Função principal ATUALIZADA com relatório completo"""

    print("🚀 INICIANDO PROCESSO COMPLETO COM CONFIRMAÇÕES...")
    print("1. Analisando ações com sistema avançado...")

    # Executar análise COMPLETA
    top_10 = analisar_mercado_com_confirmacoes(todas_acoes)

    print("2. Criando relatório detalhado...")

    # Criar conteúdo do email COMPLETO
    data_atual = datetime.now().strftime("%d/%m/%Y %H:%M")

    html_content = f"""
    <html>
    <head>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            .header {{ background: #2E86AB; color: white; padding: 20px; border-radius: 10px; }}
            .ticker {{ background: #F8F9FA; padding: 15px; margin: 10px 0; border-radius: 8px; border-left: 5px solid #2E86AB; }}
            .alta {{ border-left-color: #28A745 !important; }}
            .media {{ border-left-color: #FFC107 !important; }}
            .baixa {{ border-left-color: #DC3545 !important; }}
            .probabilidade {{ font-weight: bold; color: #28A745; }}
            .variacao-positiva {{ color: #28A745; }}
            .variacao-negativa {{ color: #DC3545; }}
            .confirmacao {{ display: inline-block; padding: 2px 8px; margin: 2px; border-radius: 4px; font-size: 12px; }}
            .confirm-ok {{ background: #28A745; color: white; }}
            .confirm-no {{ background: #6C757D; color: white; }}
            .detalhes {{ background: #E9ECEF; padding: 10px; margin: 5px 0; border-radius: 5px; font-size: 12px; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>📈 RELATÓRIO BOVESPA AVANÇADO - {data_atual}</h1>
            <p>Top 10 Ações com Análise Completa de Confirmações</p>
        </div>

        <h2>🎯 Melhores Oportunidades com Confirmações Técnicas</h2>
    """

    for i, acao in enumerate(top_10, 1):
        variacao_class = "variacao-positiva" if acao['variacao_dia'] > 0 else "variacao-negativa"

        # Definir classe baseada na probabilidade
        if acao['probabilidade_alta'] >= 70:
            classe_ticker = "alta"
        elif acao['probabilidade_alta'] >= 50:
            classe_ticker = "media"
        else:
            classe_ticker = "baixa"

        # Informações de confirmação
        detalhes = acao.get('detalhes_confirmacoes', {})

        # Volume
        vol_info = detalhes.get('volume', {})
        vol_status = "✅" if vol_info.get('confirmado') else "❌"
        vol_text = f"Volume: {vol_info.get('variacao_percentual', 'N/A')}" if vol_info.get('confirmado') else "Volume: Baixo"

        # Médias Móveis
        media_info = detalhes.get('medias_moveis', {})
        media_status = "✅" if media_info.get('confirmado') else "❌"
        media_text = media_info.get('classificacao', 'N/A')

        # Momentum (RSI)
        momentum_info = detalhes.get('momentum', {})
        momentum_status = "✅" if momentum_info.get('confirmado') else "❌"
        rsi_valor = momentum_info.get('rsi', 'N/A')
        momentum_text = f"RSI: {rsi_valor}" if rsi_valor != 'N/A' else "RSI: N/A"

        html_content += f"""
        <div class="ticker {classe_ticker}">
            <h3>#{i} - {acao['ticker']}</h3>
            <p><strong>Preço:</strong> R$ {acao['preco_atual']} |
               <strong>Variação:</strong> <span class="{variacao_class}">{acao['variacao_dia']}%</span> |
               <strong>Probabilidade:</strong> <span class="probabilidade">{acao['probabilidade_alta']}%</span></p>

            <p><strong>Padrão:</strong> {acao['padrao_detectado']} |
               <strong>Confirmações:</strong> {acao['confirmacoes']}/3</p>

            <div class="detalhes">
                <p><strong>🔍 Confirmações Técnicas:</strong></p>
                <p>{vol_status} {vol_text} | {media_status} Médias: {media_text} | {momentum_status} {momentum_text}</p>
        """

        # Detalhes expandidos se disponíveis
        if media_info.get('valores'):
            html_content += f"<p>📊 Médias: 9p={media_info['valores'].get('MA9', 'N/A')} | 21p={media_info['valores'].get('MA21', 'N/A')} | 50p={media_info['valores'].get('MA50', 'N/A')}</p>"

        html_content += """
            </div>
        </div>
        """

    html_content += """
        <br>
        <div style="background: #D4EDDA; padding: 15px; border-radius: 8px;">
            <h3>📋 Legenda do Sistema:</h3>
            <p><strong>✅ Volume:</strong> Confirmação por volume acima da média</p>
            <p><strong>✅ Médias:</strong> Alinhamento altista das médias móveis</p>
            <p><strong>✅ RSI:</strong> Momentum positivo ou saindo de oversold</p>
            <p><strong>🎯 Probabilidade:</strong> Calculada com base em padrões + confirmações</p>
        </div>

        <hr>
        <p><em>Relatório gerado automaticamente - Análise técnica com confirmações múltiplas</em></p>
    </body>
    </html>
    """

    print("3. Enviando relatório completo...")

    try:
        # Configurar servidor SMTP do Gmail
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()

        # ⚠️ IMPORTANTE: Use a senha de app do Gmail aqui
        server.login(email_from, email_password)

        # Criar mensagem
        msg = MIMEMultipart()
        msg['From'] = email_from
        msg['To'] = email_to
        msg['Subject'] = f"📊 Relatório Bovespa Completo - {datetime.now().strftime('%d/%m/%Y')}"

        # Adicionar conteúdo HTML
        msg.attach(MIMEText(html_content, 'html'))

        # Enviar email
        server.send_message(msg)
        server.quit()

        print("✅ RELATÓRIO COMPLETO ENVIADO COM SUCESSO!")
        print(f"📧 Para: {email_to}")
        print(f"📊 {len(top_10)} oportunidades analisadas com confirmações técnicas")

        # Mostrar preview no Colab
        print("\n📋 PREVIEW DO RELATÓRIO AVANÇADO:")
        for i, acao in enumerate(top_10, 1):
            confirmacoes = acao['confirmacoes']
            emoji_conf = "🔴" if confirmacoes == 0 else "🟡" if confirmacoes == 1 else "🟢"
            print(f"{i}. {acao['ticker']} - {acao['probabilidade_alta']}% - {emoji_conf}{confirmacoes}/3 confirmações")

    except Exception as e:
        print(f"❌ ERRO AO ENVIAR EMAIL: {e}")
        print("💡 Verifique a senha de app do Gmail!")

print("✅ Relatório completo configurado!")

📊 CONFIGURANDO RELATÓRIO COMPLETO COM CONFIRMAÇÕES
✅ Relatório completo configurado!


In [ ]:
# CÉLULA 14 - EXECUTAR SISTEMA COMPLETO ATUALIZADO
print("🎯 EXECUTANDO SISTEMA COMPLETO COM RELATÓRIO AVANÇADO")

# Esta função vai usar TUDO que configuramos
enviar_relatorio_completo()

print("\n✨ SISTEMA ATUALIZADO FINALIZADO!")
print("📧 Você receberá um email com:")
print("   ✅ Probabilidade de alta calculada")
print("   ✅ Confirmação por Volume")
print("   ✅ Análise de Médias Móveis")
print("   ✅ Indicador RSI (Momentum)")
print("   ✅ Padrões de candlestick")
print("   ✅ Pontuação de confirmações (0-3)")

🎯 EXECUTANDO SISTEMA COMPLETO COM RELATÓRIO AVANÇADO
🚀 INICIANDO PROCESSO COMPLETO COM CONFIRMAÇÕES...
1. Analisando ações com sistema avançado...


NameError: name 'todas_acoes' is not defined

In [ ]:
# CÉLULA 23 - RELATÓRIO DE EMAIL COM MACD E PERFORMANCE
print("📊 ATUALIZANDO RELATÓRIO COM MACD E ESTATÍSTICAS")

def enviar_relatorio_completo_com_macd():
    """Versão FINAL com MACD e estatísticas de performance"""

    print("🚀 INICIANDO SISTEMA COMPLETO COM MACD...")
    print("1. Analisando com todas as confirmações...")

    # Usar análise COMPLETA com MACD
    top_oportunidades = analisador_final.analisar_com_tracking(todas_acoes, salvar_sinais=True)

    # Verificar estatísticas de performance
    estatisticas = analisador_final.verificar_resultados_anteriores()

    print("2. Criando relatório avançado...")

    data_atual = datetime.now().strftime("%d/%m/%Y %H:%M")

    html_content = f"""
    <html>
    <head>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            .header {{ background: #2E86AB; color: white; padding: 20px; border-radius: 10px; }}
            .stats {{ background: #17A2B8; color: white; padding: 15px; border-radius: 8px; margin: 10px 0; }}
            .ticker {{ background: #F8F9FA; padding: 15px; margin: 10px 0; border-radius: 8px; border-left: 5px solid; }}
            .alta {{ border-left-color: #28A745 !important; background: #F8FFF9 !important; }}
            .media {{ border-left-color: #FFC107 !important; background: #FFFEF0 !important; }}
            .baixa {{ border-left-color: #DC3545 !important; }}
            .probabilidade {{ font-weight: bold; }}
            .alta-num {{ color: #28A745; }}
            .media-num {{ color: #FFC107; }}
            .baixa-num {{ color: #DC3545; }}
            .variacao-positiva {{ color: #28A745; font-weight: bold; }}
            .variacao-negativa {{ color: #DC3545; }}
            .detalhes {{ background: #E9ECEF; padding: 10px; margin: 5px 0; border-radius: 5px; font-size: 12px; }}
            .confirm-badge {{ display: inline-block; padding: 2px 8px; margin: 2px; border-radius: 12px; font-size: 11px; }}
            .confirm-4 {{ background: #28A745; color: white; }}
            .confirm-3 {{ background: #20C997; color: white; }}
            .confirm-2 {{ background: #FFC107; color: black; }}
            .confirm-1 {{ background: #FD7E14; color: white; }}
            .confirm-0 {{ background: #6C757D; color: white; }}
            .macd-info {{ background: #E7F3FF; padding: 8px; margin: 5px 0; border-radius: 5px; font-size: 11px; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>📈 RELATÓRIO BOVESPA AVANÇADO - {data_atual}</h1>
            <p>Sistema Completo: Padrões + MACD + Performance Tracking</p>
        </div>

        <div class="stats">
            <h3>📊 ESTATÍSTICAS DO SISTEMA</h3>
            <p>Assertividade: <strong>{estatisticas['assertividade']}%</strong> |
               Total Sinais: <strong>{estatisticas['total_sinais']}</strong> |
               Acertos: <strong>{estatisticas['acertos']}</strong> |
               Pendentes: <strong>{estatisticas['sinais_pendentes']}</strong></p>
        </div>

        <h2>🎯 TOP OPORTUNIDADES COM ANÁLISE MACD</h2>
    """

    if not top_oportunidades:
        html_content += """
        <div style="background: #FFF3CD; padding: 20px; border-radius: 8px; text-align: center;">
            <h3>⚠️ Nenhuma oportunidade forte identificada</h3>
            <p>Recomenda-se aguardar melhores condições de mercado.</p>
        </div>
        """
    else:
        for i, acao in enumerate(top_oportunidades, 1):
            # Definir classes
            if acao['probabilidade_alta'] >= 60:
                classe_ticker = "alta"
                classe_prob = "alta-num"
                emoji = "🔥"
            elif acao['probabilidade_alta'] >= 45:
                classe_ticker = "media"
                classe_prob = "media-num"
                emoji = "📈"
            else:
                classe_ticker = "baixa"
                classe_prob = "baixa-num"
                emoji = "⚡"

            variacao_class = "variacao-positiva" if acao['variacao_dia'] > 0 else "variacao-negativa"

            # Badge de confirmações (agora 4 possíveis)
            confirm_class = f"confirm-{acao['confirmacoes']}"
            confirm_text = f"{acao['confirmacoes']}/4 Confirmações"

            # Informações MACD
            detalhes_macd = acao['detalhes_confirmacoes'].get('macd', {})
            macd_info = detalhes_macd.get('detalhes', {})

            html_content += f"""
            <div class="ticker {classe_ticker}">
                <h3>{emoji} #{i} - {acao['ticker']}</h3>
                <p><strong>Preço:</strong> R$ {acao['preco_atual']} |
                   <strong>Variação:</strong> <span class="{variacao_class}">{acao['variacao_dia']}%</span> |
                   <strong>Probabilidade:</strong> <span class="probabilidade {classe_prob}">{acao['probabilidade_alta']}%</span></p>

                <p><strong>Padrão:</strong> {acao['padrao_detectado'].upper()} |
                   <span class="confirm-badge {confirm_class}">{confirm_text}</span></p>

                <p><strong>Classificação:</strong> {acao['classificacao']}</p>
            """

            # DETALHES TÉCNICOS EXPANDIDOS
            html_content += """
                <div class="detalhes">
                    <p><strong>🔍 Análise Técnica Detalhada:</strong></p>
            """

            # Volume
            vol_info = acao['detalhes_confirmacoes'].get('volume', {})
            if vol_info.get('confirmado'):
                html_content += f"<p>✅ <strong>Volume:</strong> {vol_info.get('variacao_percentual', 'N/A')} acima da média</p>"
            else:
                html_content += f"<p>❌ <strong>Volume:</strong> {vol_info.get('variacao_percentual', 'N/A')} (abaixo do esperado)</p>"

            # Médias Móveis
            media_info = acao['detalhes_confirmacoes'].get('medias_moveis', {})
            if media_info.get('confirmado'):
                valores = media_info.get('valores', {})
                html_content += f"<p>✅ <strong>Médias:</strong> {media_info.get('classificacao', '')}"
                if valores:
                    html_content += f" (9p: {valores.get('MA9', '')} | 21p: {valores.get('MA21', '')} | 50p: {valores.get('MA50', '')})"
                html_content += "</p>"

            # RSI
            rsi_info = acao['detalhes_confirmacoes'].get('momentum', {})
            if rsi_info.get('confirmado'):
                html_content += f"<p>✅ <strong>RSI:</strong> {rsi_info.get('rsi', 'N/A')} - {rsi_info.get('classificacao', '')}</p>"

            # MACD - NOVO!
            if detalhes_macd.get('confirmado'):
                html_content += f"""
                <div class="macd-info">
                    <p>✅ <strong>MACD:</strong> {macd_info.get('classificacao', 'Confirmado')}</p>
                    <p>📊 MACD Line: {macd_info.get('macd_line', 'N/A')} |
                       Signal: {macd_info.get('signal_line', 'N/A')} |
                       Histogram: {macd_info.get('histogram', 'N/A')}</p>
                    <p>🎯 Cruzamento: {('✅ SIM' if macd_info.get('cruzamento_alta') else '❌ NÃO')} |
                       Acima Zero: {('✅ SIM' if macd_info.get('acima_zero') else '❌ NÃO')}</p>
                </div>
                """
            else:
                html_content += f"<p>❌ <strong>MACD:</strong> {detalhes_macd.get('detalhes', {}).get('classificacao', 'Não confirmado')}</p>"

            html_content += """
                </div>
            </div>
            """

    html_content += """
        <br>
        <div style="background: #D1ECF1; padding: 15px; border-radius: 8px;">
            <h3>📋 LEGENDA TÉCNICA AVANÇADA:</h3>
            <p><strong>🎯 CONFIRMAÇÕES TÉCNICAS (4):</strong></p>
            <p><span class="confirm-badge confirm-4">4/4</span> <strong>EXCELENTE:</strong> Volume + Médias + RSI + MACD</p>
            <p><span class="confirm-badge confirm-3">3/4</span> <strong>MUITO BOM:</strong> 3 confirmações técnicas</p>
            <p><span class="confirm-badge confirm-2">2/4</span> <strong>BOM:</strong> 2 confirmações técnicas</p>
            <p><span class="confirm-badge confirm-1">1/4</span> <strong>REGULAR:</strong> 1 confirmação técnica</p>
            <br>
            <p><strong>📊 MACD:</strong> Indicador de momentum - confirma tendências</p>
            <p><strong>📈 ESTRATÉGIA:</strong> Operar preferencialmente com 2+ confirmações</p>
        </div>

        <hr>
        <p><em>⚡ Relatório com MACD e Tracking - Assertividade: {estatisticas['assertividade']}%</em></p>
        <p><em>📊 {len(top_oportunidades)} oportunidades de 83 ações analisadas</em></p>
    </body>
    </html>
    """.format(estatisticas=estatisticas, len=len(top_oportunidades))

    print("3. Enviando relatório com MACD...")

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(email_from, email_password)

        msg = MIMEMultipart()
        msg['From'] = email_from
        msg['To'] = email_to
        msg['Subject'] = f"📊 Relatório Bovespa com MACD - {datetime.now().strftime('%d/%m/%Y')}"

        msg.attach(MIMEText(html_content, 'html'))
        server.send_message(msg)
        server.quit()

        print("✅ RELATÓRIO COM MACD ENVIADO!")
        print(f"📧 Para: {email_to}")
        print(f"📊 {len(top_oportunidades)} oportunidades analisadas")
        print(f"🎯 Assertividade do sistema: {estatisticas['assertividade']}%")

        # Preview
        print("\n📋 RESUMO COM MACD:")
        for i, acao in enumerate(top_oportunidades, 1):
            tem_macd = "✅" if acao['detalhes_confirmacoes'].get('macd', {}).get('confirmado') else "❌"
            print(f"{i}. {acao['ticker']} - {acao['probabilidade_alta']}% - MACD:{tem_macd} - {acao['confirmacoes']}/4 confs")

    except Exception as e:
        print(f"❌ ERRO: {e}")

print("✅ Relatório com MACD configurado!")

In [ ]:
# CÉLULA 15 - OTIMIZAR SISTEMA PARA MERCADO BRASILEIRO
print("🔧 OTIMIZANDO SISTEMA PARA BOVESPA")

class AnalisadorOtimizado:
    def __init__(self):
        # Ajustar parâmetros para realidade brasileira
        self.volume_minimo = 1.1  # Reduzir para 10% (era 20%)
        self.rsi_sobrevenda = 35  # Aumentar oversold (era 30)
        self.periodo_dados = '60d'  # Aumentar período (era 30d)

    def analisar_mercado_otimizado(self, acoes):
        """Versão otimizada para mercado brasileiro"""
        print("🔍 Iniciando análise OTIMIZADA...")
        resultados = []

        for i, ticker in enumerate(acoes):
            try:
                acao = yf.Ticker(ticker)
                hist = acao.history(period=self.periodo_dados)  # Mais dados

                if len(hist) < 21:
                    continue

                preco_atual = hist['Close'].iloc[-1]
                preco_anterior = hist['Close'].iloc[-2]
                variacao = ((preco_atual - preco_anterior) / preco_anterior) * 100

                # Detectar padrões
                padrao_detectado = None
                confianca_base = 0.4  # Base mais baixa

                for padrao_nome, detector in analisador.padroes_alta.items():
                    if detector(hist):
                        padrao_detectado = padrao_nome
                        confianca_base = analisador.confiabilidade_padroes[padrao_nome]
                        break

                # Aplicar confirmações
                if padrao_detectado:
                    confianca_final, detalhes_confirmacoes, num_confirmacoes = sistema_confirmacao.analisar_confirmacoes(
                        hist, padrao_detectado, confianca_base
                    )
                else:
                    # SEM padrão: dar chance baseada em tendência
                    tendencia = (hist['Close'].iloc[-1] - hist['Close'].iloc[-5]) / hist['Close'].iloc[-5]
                    if tendencia > 0:
                        confianca_final = 0.35  # Pouco acima do neutro
                    else:
                        confianca_final = 0.25  # Abaixo do neutro
                    detalhes_confirmacoes = {}
                    num_confirmacoes = 0

                # Classificação MAIS PERMISSIVA
                if confianca_final >= 0.6:
                    classificacao = "ALTA PROBABILIDADE"
                elif confianca_final >= 0.45:
                    classificacao = "PROVÁVEL ALTA"
                elif confianca_final >= 0.35:
                    classificacao = "NEUTRO TENDENDO ALTA"
                else:
                    classificacao = "BAIXA PROBABILIDADE"

                info_acao = {
                    'ticker': ticker,
                    'preco_atual': round(preco_atual, 2),
                    'variacao_dia': round(variacao, 2),
                    'probabilidade_alta': round(confianca_final * 100, 1),
                    'classificacao': classificacao,
                    'padrao_detectado': padrao_detectado or 'Nenhum',
                    'confirmacoes': num_confirmacoes,
                    'detalhes_confirmacoes': detalhes_confirmacoes
                }

                resultados.append(info_acao)

                if (i + 1) % 10 == 0:
                    print(f"📊 Analisadas {i + 1}/{len(acoes)} ações...")

                time.sleep(0.1)  # Evitar bloqueio

            except Exception as e:
                # Ignorar erros de ações sem dados
                continue

        # Filtrar apenas ações com alguma probabilidade
        resultados_filtrados = [r for r in resultados if r['probabilidade_alta'] > 35]
        resultados_ordenados = sorted(resultados_filtrados, key=lambda x: x['probabilidade_alta'], reverse=True)
        top_10 = resultados_ordenados[:10]

        print(f"✅ Análise OTIMIZADA concluída! {len(resultados)} processadas, {len(resultados_filtrados)} com potencial")
        return top_10

# Instanciar analisador otimizado
analisador_otimizado = AnalisadorOtimizado()
print("✅ Sistema otimizado carregado!")

In [ ]:
# CÉLULA EXTRA - CARREGAR LISTA DE AÇÕES
print("📋 CARREGANDO LISTA DE AÇÕES DA BOVESPA")

def carregar_acoes_bovespa():
    """Carrega lista completa de ações da Bovespa"""
    acoes_ibovespa = [
        'ABEV3.SA', 'ALPA4.SA', 'AMER3.SA', 'ASAI3.SA', 'AZUL4.SA', 'B3SA3.SA', 'BBAS3.SA', 'BBDC3.SA', 'BBDC4.SA',
        'BBSE3.SA', 'BEEF3.SA', 'BPAC11.SA', 'BRAP4.SA', 'BRDT3.SA', 'BRFS3.SA', 'BRKM5.SA', 'BRML3.SA', 'CASH3.SA',
        'CCRO3.SA', 'CIEL3.SA', 'CMIG4.SA', 'COGN3.SA', 'CPFE3.SA', 'CPLE6.SA', 'CSAN3.SA', 'CSNA3.SA', 'CVCB3.SA',
        'CYRE3.SA', 'ECOR3.SA', 'EGIE3.SA', 'ELET3.SA', 'ELET6.SA', 'EMBR3.SA', 'ENBR3.SA', 'ENEV3.SA', 'ENGI11.SA',
        'EQTL3.SA', 'EZTC3.SA', 'FLRY3.SA', 'GGBR4.SA', 'GOAU4.SA', 'GOLL4.SA', 'HAPV3.SA', 'HYPE3.SA', 'IGTA3.SA',
        'IRBR3.SA', 'ITSA4.SA', 'ITUB4.SA', 'JBSS3.SA', 'JHSF3.SA', 'KLBN11.SA', 'LREN3.SA', 'LWSA3.SA', 'MGLU3.SA',
        'MRFG3.SA', 'MRVE3.SA', 'MULT3.SA', 'NTCO3.SA', 'PCAR3.SA', 'PETR3.SA', 'PETR4.SA', 'PRIO3.SA', 'QUAL3.SA',
        'RADL3.SA', 'RAIL3.SA', 'RDOR3.SA', 'RENT3.SA', 'RRRP3.SA', 'SANB11.SA', 'SBSP3.SA', 'SLCE3.SA', 'SOMA3.SA',
        'SUZB3.SA', 'TAEE11.SA', 'TIMS3.SA', 'TOTS3.SA', 'UGPA3.SA', 'USIM5.SA', 'VALE3.SA', 'VBBR3.SA', 'VIVT3.SA',
        'WEGE3.SA', 'YDUQ3.SA'
    ]

    print(f"📊 Total de ações carregadas: {len(acoes_ibovespa)}")
    return acoes_ibovespa

# Carregar as ações
todas_acoes = carregar_acoes_bovespa()
print("✅ Lista de ações carregada com sucesso!")

In [ ]:
# CÉLULA 18 - EXECUTAR SISTEMA PROFISSIONAL FINAL
print("🎯 EXECUTANDO SISTEMA PROFISSIONAL FINAL")

enviar_relatorio_final_otimizado()

print("\n✨ SISTEMA PROFISSIONAL CONCLUÍDO!")
print("📊 Seu relatório agora inclui:")
print("   ✅ Probabilidades REALISTAS (40-60%)")
print("   ✅ Padrões de candlestick CONFIRMADOS")
print("   ✅ Sistema de confirmações (0-3)")
print("   ✅ Classificação profissional")
print("   ✅ Filtro inteligente")
print("   ✅ Design otimizado para email")

In [ ]:
# CÉLULA 16 - TESTE DO SISTEMA OTIMIZADO
print("🧪 TESTANDO SISTEMA OTIMIZADO")

# Teste com ações mais líquidas primeiro
acoes_liquidas = ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'BBDC4.SA', 'WEGE3.SA', 'MGLU3.SA', 'B3SA3.SA']
resultados_otimizados = analisador_otimizado.analisar_mercado_otimizado(acoes_liquidas)

print("\n📊 RESULTADOS OTIMIZADOS:")
for i, acao in enumerate(resultados_otimizados, 1):
    emoji_conf = "🔴" if acao['confirmacoes'] == 0 else "🟡" if acao['confirmacoes'] == 1 else "🟢"
    print(f"{i}. {acao['ticker']} - {acao['probabilidade_alta']}% - {acao['classificacao']}")
    print(f"   Padrão: {acao['padrao_detectado']} | Confirmações: {emoji_conf}{acao['confirmacoes']}/3")

if resultados_otimizados:
    print(f"\n🎯 Encontradas {len(resultados_otimizados)} oportunidades!")
else:
    print("\n⚠️  Nenhuma oportunidade encontrada - vamos ajustar mais!")

In [ ]:
# CÉLULA 17 - FUNÇÃO FINAL OTIMIZADA PARA EMAIL
print("📧 CONFIGURANDO FUNÇÃO FINAL OTIMIZADA")

def enviar_relatorio_final_otimizado():
    """Versão FINAL otimizada do sistema completo"""

    print("🚀 EXECUTANDO SISTEMA COMPLETO OTIMIZADO...")
    print("1. Analisando mercado com parâmetros ajustados...")

    # Usar a análise otimizada
    top_oportunidades = analisador_otimizado.analisar_mercado_otimizado(todas_acoes)

    # Se menos de 5 oportunidades, completar com melhores disponíveis
    if len(top_oportunidades) < 5:
        print("🔍 Buscando mais oportunidades...")
        todas_analisadas = analisador_otimizado.analisar_mercado_otimizado(todas_acoes[:30])  # Ações mais líquidas
        top_oportunidades = todas_analisadas[:5]  # Top 5 no máximo

    print("2. Criando relatório final...")

    data_atual = datetime.now().strftime("%d/%m/%Y %H:%M")

    html_content = f"""
    <html>
    <head>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            .header {{ background: #2E86AB; color: white; padding: 20px; border-radius: 10px; }}
            .ticker {{ background: #F8F9FA; padding: 15px; margin: 10px 0; border-radius: 8px; border-left: 5px solid; }}
            .alta {{ border-left-color: #28A745 !important; background: #F8FFF9 !important; }}
            .media {{ border-left-color: #FFC107 !important; background: #FFFEF0 !important; }}
            .baixa {{ border-left-color: #DC3545 !important; }}
            .probabilidade {{ font-weight: bold; }}
            .alta-num {{ color: #28A745; }}
            .media-num {{ color: #FFC107; }}
            .baixa-num {{ color: #DC3545; }}
            .variacao-positiva {{ color: #28A745; font-weight: bold; }}
            .variacao-negativa {{ color: #DC3545; }}
            .detalhes {{ background: #E9ECEF; padding: 10px; margin: 5px 0; border-radius: 5px; font-size: 12px; }}
            .confirm-badge {{ display: inline-block; padding: 2px 8px; margin: 2px; border-radius: 12px; font-size: 11px; }}
            .confirm-3 {{ background: #28A745; color: white; }}
            .confirm-2 {{ background: #FFC107; color: black; }}
            .confirm-1 {{ background: #FD7E14; color: white; }}
            .confirm-0 {{ background: #6C757D; color: white; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>📈 RELATÓRIO BOVESPA PROFISSIONAL - {data_atual}</h1>
            <p>Sistema Otimizado com Análise Técnica Avançada</p>
        </div>

        <h2>🎯 TOP OPORTUNIDADES IDENTIFICADAS</h2>
    """

    if not top_oportunidades:
        html_content += """
        <div style="background: #FFF3CD; padding: 20px; border-radius: 8px; text-align: center;">
            <h3>⚠️ Nenhuma oportunidade forte identificada</h3>
            <p>O mercado pode estar em fase de consolidação. Recomenda-se aguardar novos sinais.</p>
        </div>
        """
    else:
        for i, acao in enumerate(top_oportunidades, 1):
            # Definir classes baseadas na probabilidade
            if acao['probabilidade_alta'] >= 60:
                classe_ticker = "alta"
                classe_prob = "alta-num"
                emoji = "🔥"
            elif acao['probabilidade_alta'] >= 45:
                classe_ticker = "media"
                classe_prob = "media-num"
                emoji = "📈"
            else:
                classe_ticker = "baixa"
                classe_prob = "baixa-num"
                emoji = "⚡"

            variacao_class = "variacao-positiva" if acao['variacao_dia'] > 0 else "variacao-negativa"

            # Badge de confirmações
            confirm_class = f"confirm-{acao['confirmacoes']}"
            confirm_text = f"{acao['confirmacoes']}/3 Confirmações"

            html_content += f"""
            <div class="ticker {classe_ticker}">
                <h3>{emoji} #{i} - {acao['ticker']}</h3>
                <p><strong>Preço:</strong> R$ {acao['preco_atual']} |
                   <strong>Variação:</strong> <span class="{variacao_class}">{acao['variacao_dia']}%</span> |
                   <strong>Probabilidade:</strong> <span class="probabilidade {classe_prob}">{acao['probabilidade_alta']}%</span></p>

                <p><strong>Padrão:</strong> {acao['padrao_detectado'].upper()} |
                   <span class="confirm-badge {confirm_class}">{confirm_text}</span></p>

                <p><strong>Classificação:</strong> {acao['classificacao']}</p>
            </div>
            """

    html_content += """
        <br>
        <div style="background: #D1ECF1; padding: 15px; border-radius: 8px;">
            <h3>📋 LEGENDA TÉCNICA:</h3>
            <p><span class="confirm-badge confirm-3">3/3</span> <strong>FORTE:</strong> Volume + Médias + Momentum confirmados</p>
            <p><span class="confirm-badge confirm-2">2/3</span> <strong>MODERADO:</strong> 2 confirmações técnicas</p>
            <p><span class="confirm-badge confirm-1">1/3</span> <strong>LEVE:</strong> 1 confirmação técnica</p>
            <p><span class="confirm-badge confirm-0">0/3</span> <strong>FRACO:</strong> Apenas padrão visual</p>
            <br>
            <p><strong>🎯 ESTRATÉGIA RECOMENDADA:</strong> Operar preferencialmente com 2+ confirmações</p>
        </div>

        <hr>
        <p><em>⚡ Relatório gerado automaticamente - Sistema otimizado para Bovespa</em></p>
        <p><em>📊 {len(top_oportunidades)} oportunidades filtradas de 83 ações analisadas</em></p>
    </body>
    </html>
    """

    print("3. Enviando relatório profissional...")

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(email_from, email_password)

        msg = MIMEMultipart()
        msg['From'] = email_from
        msg['To'] = email_to
        msg['Subject'] = f"🎯 Relatório Bovespa Profissional - {datetime.now().strftime('%d/%m/%Y')}"

        msg.attach(MIMEText(html_content, 'html'))
        server.send_message(msg)
        server.quit()

        print("✅ RELATÓRIO PROFISSIONAL ENVIADO!")
        print(f"📧 Para: {email_to}")
        print(f"📊 {len(top_oportunidades)} oportunidades no relatório")

        # Preview
        print("\n📋 RESUMO FINAL:")
        for i, acao in enumerate(top_oportunidades, 1):
            conf_emoji = "🟢" if acao['confirmacoes'] >= 2 else "🟡" if acao['confirmacoes'] == 1 else "🔴"
            print(f"{i}. {acao['ticker']} - {acao['probabilidade_alta']}% - {conf_emoji}{acao['confirmacoes']} confirmações - {acao['padrao_detectado']}")

    except Exception as e:
        print(f"❌ ERRO: {e}")

print("✅ Função final otimizada carregada!")

In [ ]:
# CÉLULA 19 - SISTEMA MACD + TRACKING DE ASSERTIVIDADE
print("📊 CONFIGURANDO SISTEMA MACD E HISTÓRICO DE PERFORMANCE")

import json
from datetime import datetime, timedelta

class AnalisadorMACD:
    def __init__(self):
        self.fast = 12
        self.slow = 26
        self.signal = 9

    def analisar_macd(self, df):
        """Analisa sinal MACD para confirmação"""
        if len(df) < self.slow + self.signal:
            return False, {}, "Dados insuficientes"

        try:
            # Calcular MACD
            macd_data = ta.macd(df['Close'], fast=self.fast, slow=self.slow, signal=self.signal)
            macd_line = macd_data[f'MACD_{self.fast}_{self.slow}_{self.signal}'].iloc[-1]
            signal_line = macd_data[f'MACDs_{self.fast}_{self.slow}_{self.signal}'].iloc[-1]
            histogram = macd_data[f'MACDh_{self.fast}_{self.slow}_{self.signal}'].iloc[-1]

            # Análise do MACD
            cruzamento_alta = macd_line > signal_line
            acima_zero = macd_line > 0
            histograma_positivo = histogram > 0
            histograma_crescendo = histogram > macd_data[f'MACDh_{self.fast}_{self.slow}_{self.signal}'].iloc[-2] if len(macd_data) > 1 else False

            # Pontuação MACD
            pontos = 0
            if cruzamento_alta:
                pontos += 2
            if acima_zero:
                pontos += 1
            if histograma_positivo:
                pontos += 1
            if histograma_crescendo:
                pontos += 1

            # Classificação
            if pontos >= 4:
                classificacao = "MACD FORTE ALTISTA"
                confirmado = True
            elif pontos >= 2:
                classificacao = "MACD MODERADO ALTISTA"
                confirmado = True
            else:
                classificacao = "MACD NEUTRO/BAIXISTA"
                confirmado = False

            detalhes = {
                'macd_line': round(macd_line, 4),
                'signal_line': round(signal_line, 4),
                'histogram': round(histogram, 4),
                'cruzamento_alta': cruzamento_alta,
                'acima_zero': acima_zero,
                'pontos': pontos,
                'classificacao': classificacao
            }

            return confirmado, detalhes, pontos

        except Exception as e:
            return False, {}, f"Erro MACD: {e}"

class TrackPerformance:
    def __init__(self):
        self.historico = {}
        self.arquivo_historico = "historico_performance.json"

    def salvar_sinal(self, ticker, dados_sinal, data_sinal=None):
        """Salva um sinal gerado para tracking futuro"""
        if data_sinal is None:
            data_sinal = datetime.now().strftime("%Y-%m-%d")

        sinal_id = f"{ticker}_{data_sinal}"

        self.historico[sinal_id] = {
            'ticker': ticker,
            'data_sinal': data_sinal,
            'probabilidade_inicial': dados_sinal['probabilidade_alta'],
            'padrao_detectado': dados_sinal['padrao_detectado'],
            'confirmacoes_iniciais': dados_sinal['confirmacoes'],
            'preco_entrada': dados_sinal['preco_atual'],
            'status': 'PENDENTE',  # PENDENTE, SUCESSO, FALHA
            'resultado_4dias': None,
            'variacao_4dias': None,
            'data_verificacao': None
        }

        self._salvar_arquivo()
        return sinal_id

    def verificar_resultado_4dias(self, ticker, data_sinal=None):
        """Verifica o resultado 4 dias após o sinal"""
        if data_sinal is None:
            data_sinal = datetime.now().strftime("%Y-%m-%d")

        sinal_id = f"{ticker}_{data_sinal}"

        if sinal_id not in self.historico:
            return False, "Sinal não encontrado"

        try:
            # Buscar dados dos últimos 10 dias (para ter os 4 dias futuros)
            acao = yf.Ticker(ticker)
            hist = acao.history(period='10d')

            if len(hist) < 5:
                return False, "Dados insuficientes"

            # Encontrar preços
            data_sinal_dt = datetime.strptime(data_sinal, "%Y-%m-%d")
            preco_entrada = self.historico[sinal_id]['preco_entrada']

            # Preço 4 dias depois (ou o mais próximo possível)
            if len(hist) >= 5:
                preco_4dias = hist['Close'].iloc[-1]  # Último preço disponível
            else:
                preco_4dias = hist['Close'].iloc[-1]

            # Calcular variação
            variacao = ((preco_4dias - preco_entrada) / preco_entrada) * 100

            # Determinar sucesso (>2% de ganho em 4 dias)
            sucesso = variacao > 2.0

            # Atualizar histórico
            self.historico[sinal_id].update({
                'status': 'SUCESSO' if sucesso else 'FALHA',
                'resultado_4dias': 'SUCESSO' if sucesso else 'FALHA',
                'variacao_4dias': round(variacao, 2),
                'preco_4dias': round(preco_4dias, 2),
                'data_verificacao': datetime.now().strftime("%Y-%m-%d %H:%M")
            })

            self._salvar_arquivo()

            return True, {
                'sucesso': sucesso,
                'variacao': variacao,
                'preco_entrada': preco_entrada,
                'preco_4dias': preco_4dias
            }

        except Exception as e:
            return False, f"Erro na verificação: {e}"

    def calcular_estatisticas(self):
        """Calcula estatísticas de performance"""
        if not self.historico:
            return {"total_sinais": 0, "acertos": 0, "assertividade": 0}

        total_sinais = len([s for s in self.historico.values() if s['status'] in ['SUCESSO', 'FALHA']])
        acertos = len([s for s in self.historico.values() if s['status'] == 'SUCESSO'])

        assertividade = (acertos / total_sinais * 100) if total_sinais > 0 else 0

        return {
            'total_sinais': total_sinais,
            'acertos': acertos,
            'assertividade': round(assertividade, 1),
            'sinais_pendentes': len([s for s in self.historico.values() if s['status'] == 'PENDENTE'])
        }

    def _salvar_arquivo(self):
        """Salva histórico em arquivo"""
        try:
            with open(self.arquivo_historico, 'w') as f:
                json.dump(self.historico, f, indent=2)
        except:
            pass

    def _carregar_arquivo(self):
        """Carrega histórico do arquivo"""
        try:
            with open(self.arquivo_historico, 'r') as f:
                self.historico = json.load(f)
        except:
            self.historico = {}

# Instanciar sistemas
analisador_macd = AnalisadorMACD()
tracker = TrackPerformance()
print("✅ Sistema MACD e Performance carregados!")

In [ ]:
# CÉLULA 20 - SISTEMA DE CONFIRMAÇÃO COMPLETO (COM MACD)
print("🔧 ATUALIZANDO SISTEMA DE CONFIRMAÇÃO COM MACD")

class SistemaConfirmacaoCompleto:
    def __init__(self):
        # Pesos atualizados incluindo MACD
        self.pesos = {
            'volume': 0.20,
            'medias_moveis': 0.25,
            'momentum': 0.20,
            'macd': 0.20,
            'padrao_base': 0.15
        }

    def analisar_confirmacoes_completas(self, df, padrao_detectado, confianca_base):
        """
        Analisa TODAS as confirmações incluindo MACD
        """
        print(f"🔍 Analisando confirmações completas para: {padrao_detectado}")

        detalhes = {}
        pontuacao_total = 0

        # 1. Confirmação por Volume
        try:
            volume_confirmado, vol_atual, vol_medio, variacao = confirmador_volume.analisar_volume_padrao(df, padrao_detectado)
            detalhes['volume'] = {
                'confirmado': volume_confirmado,
                'volume_atual': f"{vol_atual:,.0f}",
                'volume_medio': f"{vol_medio:,.0f}",
                'variacao_percentual': f"{variacao:+.1%}",
                'pontos': 1 if volume_confirmado else 0
            }
            pontuacao_total += detalhes['volume']['pontos'] * self.pesos['volume']
        except Exception as e:
            detalhes['volume'] = {'confirmado': False, 'erro': str(e), 'pontos': 0}

        # 2. Confirmação por Médias Móveis
        try:
            media_confirmado, valores_medias, classificacao_media, pontos_media = analisador_medias.analisar_medias(df)
            detalhes['medias_moveis'] = {
                'confirmado': media_confirmado,
                'valores': valores_medias,
                'classificacao': classificacao_media,
                'pontos': pontos_media
            }
            pontuacao_total += pontos_media * self.pesos['medias_moveis'] / 3.0
        except Exception as e:
            detalhes['medias_moveis'] = {'confirmado': False, 'erro': str(e), 'pontos': 0}

        # 3. Confirmação por Momentum (RSI)
        try:
            rsi_valor, classificacao_rsi, pontos_rsi, condicoes_rsi = analisador_momentum.calcular_rsi(df)
            detalhes['momentum'] = {
                'confirmado': pontos_rsi > 0,
                'rsi': rsi_valor,
                'classificacao': classificacao_rsi,
                'pontos': pontos_rsi
            }
            pontuacao_total += pontos_rsi * self.pesos['momentum'] / 3.0
        except Exception as e:
            detalhes['momentum'] = {'confirmado': False, 'erro': str(e), 'pontos': 0}

        # 4. NOVO: Confirmação por MACD
        try:
            macd_confirmado, detalhes_macd, pontos_macd = analisador_macd.analisar_macd(df)
            detalhes['macd'] = {
                'confirmado': macd_confirmado,
                'detalhes': detalhes_macd,
                'pontos': pontos_macd
            }
            pontuacao_total += pontos_macd * self.pesos['macd'] / 4.0
        except Exception as e:
            detalhes['macd'] = {'confirmado': False, 'erro': str(e), 'pontos': 0}

        # 5. Peso do padrão base
        pontuacao_total += confianca_base * self.pesos['padrao_base']

        # Calcular confiança final (0-100%)
        confianca_final = min(0.95, pontuacao_total)

        # Contar confirmações ativas (agora 4 possíveis)
        confirmacoes_ativas = sum([
            detalhes['volume'].get('confirmado', False),
            detalhes['medias_moveis'].get('confirmado', False),
            detalhes['momentum'].get('confirmado', False),
            detalhes['macd'].get('confirmado', False)
        ])

        return confianca_final, detalhes, confirmacoes_ativas

# Substituir o sistema anterior
sistema_confirmacao_completo = SistemaConfirmacaoCompleto()
print("✅ Sistema de confirmação completo (com MACD) carregado!")

In [ ]:
# CÉLULA 24 - EXECUTAR SISTEMA COMPLETO COM MACD
print("🎯 EXECUTANDO SISTEMA COMPLETO COM MACD")

enviar_relatorio_completo_com_macd()

print("\n✨ SISTEMA COM MACD CONCLUÍDO!")
print("📊 Agora seu relatório inclui:")
print("   ✅ Análise MACD completa")
print("   ✅ Estatísticas de performance")
print("   ✅ 4 confirmações técnicas")
print("   ✅ Tracking de assertividade")
print("   ✅ Detalhes expandidos de cada indicador")